Business Context:
- "What's the business problem we're trying to solve?"
- "Who will use this analysis and how?"
- "What does success look like for this project?"

Data Context:
- "What do we know about the data quality?"
- "Are there any known data issues I should be aware of?"
- "What's the expected size of the dataset?"
- "Are there any performance constraints?"

Requirements Clarification:
- "What are the key outputs/deliverables?"
- "Are there any specific metrics or KPIs to focus on?"
- "What's the timeline and how does this fit into the larger pipeline?"
```

Assumptions:
- jfjf

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [46]:
df = pd.read_csv('datasets/exercise_00/sample_sales.csv')
display(df.head())
display(df.info())

,transaction_id,transaction_date,customer_id,product_category,product_name,quantity,unit_price,total_amount,sales_rep,region
0,TXN_000001,2025-04-20,CUST_0075,Home,Tools,1,400.86,400.86,Bob Smith,Central
1,TXN_000002,2024-10-11,CUST_0089,Clothing,Jacket,1,63.03,63.03,David Wilson,Central
2,TXN_000003,2025-02-07,CUST_0051,Books,Magazine,2,51.47,102.94,Eva Brown,East
3,TXN_000004,2024-08-20,CUST_0008,Sports,Apparel,2,148.84,297.68,Eva Brown,West
4,TXN_000005,2025-04-17,CUST_0190,Home,Tools,4,398.81,1595.24,Bob Smith,South


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    1000 non-null   object 
 1   transaction_date  1000 non-null   object 
 2   customer_id       1000 non-null   object 
 3   product_category  1000 non-null   object 
 4   product_name      1000 non-null   object 
 5   quantity          1000 non-null   int64  
 6   unit_price        1000 non-null   float64
 7   total_amount      1000 non-null   float64
 8   sales_rep         982 non-null    object 
 9   region            991 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 78.3+ KB


None

In [47]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

In [48]:
nullSales = df.loc[pd.isna(df['sales_rep'])]
nullSales

,transaction_id,transaction_date,customer_id,product_category,product_name,quantity,unit_price,total_amount,sales_rep,region
10,TXN_000011,2024-12-13,CUST_0003,Electronics,Laptop,1,968.79,968.79,NaN,West
173,TXN_000174,2025-04-15,CUST_0168,Books,Textbook,2,18.84,37.68,NaN,South
240,TXN_000241,2025-04-09,CUST_0113,Books,Textbook,1,57.18,57.18,NaN,West
325,TXN_000326,2025-01-31,CUST_0025,Clothing,Shirt,2,96.92,193.84,NaN,West
331,TXN_000332,2025-04-25,CUST_0097,Sports,Supplements,3,204.12,612.36,NaN,East
422,TXN_000423,2025-05-24,CUST_0050,Clothing,Shirt,1,76.80,76.80,NaN,East
467,TXN_000468,2024-09-06,CUST_0155,Books,Magazine,2,56.50,114.28,NaN,East
477,TXN_000478,2025-02-08,CUST_0105,Sports,Shoes,4,42.04,168.16,NaN,North
516,TXN_000517,2024-08-09,CUST_0064,Electronics,Headphones,1,1266.49,1266.49,NaN,South
547,TXN_000548,2024-09-01,CUST_0179,Home,Kitchen,1,223.74,223.74,NaN,East


In [49]:
df.loc[df['region'] == 'West', 'sales_rep'].value_counts()

sales_rep
Eva Brown        46
Bob Smith        40
Carol Davis      39
Alice Johnson    37
David Wilson     33
Name: count, dtype: int64

In [50]:
topRepsByRegion = df.groupby('region')['sales_rep'].agg(lambda x: x.value_counts().index[0])

In [58]:
for region in nullSales['region'].unique():
    nullSales.loc[nullSales['region'] == region, 'sales_rep'] = topRepsByRegion.loc[region]

In [59]:
nullSales

,transaction_id,transaction_date,customer_id,product_category,product_name,quantity,unit_price,total_amount,sales_rep,region
10,TXN_000011,2024-12-13,CUST_0003,Electronics,Laptop,1,968.79,968.79,Eva Brown,West
173,TXN_000174,2025-04-15,CUST_0168,Books,Textbook,2,18.84,37.68,Alice Johnson,South
240,TXN_000241,2025-04-09,CUST_0113,Books,Textbook,1,57.18,57.18,Eva Brown,West
325,TXN_000326,2025-01-31,CUST_0025,Clothing,Shirt,2,96.92,193.84,Eva Brown,West
331,TXN_000332,2025-04-25,CUST_0097,Sports,Supplements,3,204.12,612.36,David Wilson,East
422,TXN_000423,2025-05-24,CUST_0050,Clothing,Shirt,1,76.80,76.80,David Wilson,East
467,TXN_000468,2024-09-06,CUST_0155,Books,Magazine,2,56.50,114.28,David Wilson,East
477,TXN_000478,2025-02-08,CUST_0105,Sports,Shoes,4,42.04,168.16,David Wilson,North
516,TXN_000517,2024-08-09,CUST_0064,Electronics,Headphones,1,1266.49,1266.49,Alice Johnson,South
547,TXN_000548,2024-09-01,CUST_0179,Home,Kitchen,1,223.74,223.74,David Wilson,East


In [61]:
df.loc[pd.isna(df['region'])]

,transaction_id,transaction_date,customer_id,product_category,product_name,quantity,unit_price,total_amount,sales_rep,region
78,TXN_000079,2025-07-15,CUST_0131,Electronics,Laptop,2,1276.91,2553.82,Carol Davis,NaN
245,TXN_000246,2024-09-12,CUST_0100,Books,Non-fiction,1,57.04,57.04,Carol Davis,NaN
436,TXN_000437,2024-09-11,CUST_0081,Sports,Shoes,2,79.37,158.74,Carol Davis,NaN
534,TXN_000535,2025-07-06,CUST_0158,Books,Fiction,1,66.62,66.62,Alice Johnson,NaN
555,TXN_000556,2024-08-14,CUST_0043,Electronics,Laptop,3,464.63,1393.89,Bob Smith,NaN
595,TXN_000596,2024-09-15,CUST_0037,Sports,Apparel,1,36.22,36.22,Carol Davis,NaN
633,TXN_000634,2025-05-05,CUST_0015,Electronics,Headphones,1,1785.17,1785.17,Carol Davis,NaN
919,TXN_000920,2025-03-11,CUST_0167,Home,Kitchen,3,122.18,366.54,Carol Davis,NaN
993,TXN_000994,2024-12-12,CUST_0159,Clothing,Hat,3,101.80,305.40,Alice Johnson,NaN


In [65]:
df.groupby('region')['customer_id'].agg(lambda x: display(x.value_counts()))

customer_id
CUST_0179    4
CUST_0182    4
CUST_0081    4
CUST_0082    4
CUST_0164    3
            ..
CUST_0050    1
CUST_0162    1
CUST_0141    1
CUST_0104    1
CUST_0146    1
Name: count, Length: 125, dtype: int64

customer_id
CUST_0098    5
CUST_0007    4
CUST_0099    4
CUST_0184    3
CUST_0183    3
            ..
CUST_0082    1
CUST_0186    1
CUST_0165    1
CUST_0032    1
CUST_0071    1
Name: count, Length: 126, dtype: int64

customer_id
CUST_0158    5
CUST_0039    3
CUST_0066    3
CUST_0165    3
CUST_0040    3
            ..
CUST_0002    1
CUST_0136    1
CUST_0007    1
CUST_0029    1
CUST_0169    1
Name: count, Length: 128, dtype: int64

customer_id
CUST_0161    4
CUST_0127    4
CUST_0123    4
CUST_0007    4
CUST_0160    3
            ..
CUST_0112    1
CUST_0011    1
CUST_0035    1
CUST_0036    1
CUST_0147    1
Name: count, Length: 117, dtype: int64

customer_id
CUST_0004    5
CUST_0014    4
CUST_0036    4
CUST_0084    3
CUST_0165    3
            ..
CUST_0029    1
CUST_0167    1
CUST_0031    1
CUST_0172    1
CUST_0094    1
Name: count, Length: 126, dtype: int64

region
Central    None
East       None
North      None
South      None
West       None
Name: customer_id, dtype: object